# Libraries

In [2]:
import os
import requests
import json

# Get token programmmatically

In [ ]:

def oAuth_example():

  url = "https://oauth2.bitquery.io/oauth2/token"

  payload = 'grant_type=client_credentials&client_id=b5d0cf60-576d-4edc-a04f-779f6022470a&client_secret=e1WJBc6tYEPD9Z2CEo~1hektZy&scope=api'

  headers = {'Content-Type': 'application/x-www-form-urlencoded'}

  response = requests.request("POST", url, headers=headers, data=payload)
  resp = json.loads(response.text)
  print(resp)
  access_token=resp['access_token']

  url_graphql = "https://streaming.bitquery.io/graphql"
  headers_graphql = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {access_token}'
  }



oAuth_example()

{'access_token': 'ory_at_jgssi6zitf7IgoG1wbccsZYAUWp5kqi8NFUVAHQlQtI.HKttVtk7Rf_NQt3aQiHEGMcx2d-VpIRj6vNZnQmiceg', 'expires_in': 31553999, 'scope': 'api', 'token_type': 'bearer'}


In [3]:
API_KEY = os.getenv('API_KEY')

In [4]:
API_KEY

'Bearer ory_at_bsdgYItUpSmMah9ahPhmDZYLE2xSDwnwOpeFye2Vigo.kt8gzV0EgDsaxmFz8Z9RQEs_oejv7fenu6AwdBtl5mI'

In [18]:
from datetime import date
today = date.today()

In [19]:
today

datetime.date(2025, 2, 6)

In [17]:
query = """{
  EVM(dataset: combined, network: arbitrum) {
    Transfers(
      where: {
        Transaction: {
          To: { is: "0xe35e9842fceaca96570b734083f4a58e8f7c5f2a" }
        }
        Block: { Date: { is: "2024-12-12" } }
      }
      limit: { count: 10 }
      orderBy: { descending: Block_Number }
    ) {
      Transfer {
        Amount
        Currency {
          Name
          Symbol
          SmartContract
        }
        Sender
        Receiver
      }
      Call {
        Signature {
          Name
        }
        From
        To
        Value
        CallPath
      }
      Block {
        Number
        Time
      }
    }
  }
}
"""
url = "https://streaming.bitquery.io/graphql"

payload = json.dumps({
   "query": query,

   "variables": "{}"
})
headers = {
   'Content-Type': 'application/json',
   'Authorization': API_KEY
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"data":{"EVM":{"Transfers":[{"Block":{"Number":"284143373","Time":"2024-12-12T23:59:30Z"},"Call":{"CallPath":[0,2],"From":"0xe35e9842fceaca96570b734083f4a58e8f7c5f2a","Signature":{"Name":""},"To":"0xb2d26066c4a141784ef08d063d09f5b25546d8d3","Value":"9061914606197072"},"Transfer":{"Amount":"0.009061914606197072","Currency":{"Name":"Ethereum","SmartContract":"0x","Symbol":"ETH"},"Receiver":"0xb2d26066c4a141784ef08d063d09f5b25546d8d3","Sender":"0xe35e9842fceaca96570b734083f4a58e8f7c5f2a"}},{"Block":{"Number":"284143373","Time":"2024-12-12T23:59:30Z"},"Call":{"CallPath":[0,0,0],"From":"0x82af49447d8a07e3bd95bd0d56f35241523fbab1","Signature":{"Name":"transferFrom"},"To":"0x8b194beae1d3e0788a1a35173978001acdfba668","Value":"0"},"Transfer":{"Amount":"0.009061914606197072","Currency":{"Name":"Wrapped Ether","SmartContract":"0x82af49447d8a07e3bd95bd0d56f35241523fbab1","Symbol":"WETH"},"Receiver":"0xe35e9842fceaca96570b734083f4a58e8f7c5f2a","Sender":"0x3d7dc36aa2b542ad239012730dfdb23f03d75be9"}

In [6]:
import requests


def run_query(query, api_key):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': api_key}
    request = requests.post(url,
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code,
                        request.text))

In [7]:

# The GraphQL query

query = """
    query{
      bitcoin{
        blocks{
          count
        }
      }
    }
"""
result = run_query(query, api_key=API_KEY)  # Execute the query
print ('Result - {}'.format(result))

Exception: Query failed and return code is 401.      Unauthorized. You have to use Authorization or X-API-KEY tokens as described in the documentation https://docs.bitquery.io/docs/category/authorization

In [14]:
import websocket
import json

BITQUERY_API_KEY = API_KEY  # Replace with your Bitquery V2 API key

def on_message(ws, message):
    data = json.loads(message)
    print("New Transaction:")
    print(json.dumps(data, indent=4))  # Pretty-print the JSON response

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("Closed:", close_status_code, close_msg)

def on_open(ws):
    print("Connected to Bitquery WebSocket")
    query = {
        "type": "subscribe",
        "query": {
            "network": "ethereum",  # Change to "bsc", "polygon", etc.
            "limit": 10  # Adjust the limit of transactions to receive at a time
        }
    }
    ws.send(json.dumps(query))  # Send subscription request

ws = websocket(
    "wss://streaming.bitquery.io/trades",
    header={"X-API-KEY": BITQUERY_API_KEY},
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

ws.on_open = on_open
ws.run_forever()


TypeError: 'module' object is not callable